In [49]:
import os
os.environ["DEEPLAKE_DOWNLOAD_PATH"] = "./dataset"

In [50]:
import deeplake
from torchvision import transforms
from pprint import pprint

ds = deeplake.dataset('hub://activeloop/wiki-art', access_method = 'local')

filtered_labels = ['realism', 'impressionism', 'romanticism']
@deeplake.compute
def filter_labels(sample_in, labels_list):
    return sample_in.labels.data()['text'][0] in labels_list

ds_view = ds.filter(filter_labels(filtered_labels), scheduler = 'threaded', num_workers = 0)
print("Unfiltered len:", len(ds))
print("Filtered len:", len(ds_view))

transformation = transforms.Compose([
    transforms.ToTensor(),
    # TODO: figure out these values
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),

    # TODO: figure out a reasonable value here
    transforms.CenterCrop(32)
])

wikiart = ds_view.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

./dataset\hub_activeloop_wiki-art loaded successfully.
** Loaded local copy of dataset. Downloaded on: Wed Dec 14 00:48:59 2022


100%|██████████| 81433/81433 [00:02<00:00, 27703.29it/s]
c:\Users\ramon\Developer\.venv\lib\site-packages\deeplake\integrations\pytorch\common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Unfiltered len: 81433
Filtered len: 30808


In [54]:
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 27)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


def training_function(train_loader, model, loss_fn, optimizer, device, n_epochs):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            labels = labels.flatten() # this is needed because for some reason every label is in a seperate tensor
            # print(labels)
            # print(imgs.shape)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 5 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


cnn32 = Net(32)

In [57]:
import torch
import torch.optim as optim

device = (torch.device('cuda') if torch.cuda.is_available()
                      else torch.device('cpu'))

print(f"Training on {device}:")
print(f"\tdevice: {torch.cuda.get_device_name(0)}")
print("\tSize of wikiart:", len(wikiart))

training_function(
    train_loader = wikiart,
    model = cnn32.to(device=device),
    loss_fn = nn.NLLLoss(),
    optimizer = optim.SGD(cnn32.parameters(), lr=1e-2),
    device=device,
    n_epochs = 2,
)

Training on cuda:	device: NVIDIA GeForce RTX 3060 Ti
	Size of wikiart: 963
